In [1]:
!nvidia-smi

Wed Sep 11 23:51:41 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.81                 Driver Version: 560.81         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070      WDDM  |   00000000:01:00.0  On |                  N/A |
| 56%   38C    P0             30W /  215W |    1222MiB /  12282MiB |      4%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import tensorflow as tf
from helper_functions import unzip_data, create_tensorboard_callback,plot_loss_curves,compare_historys

In [3]:
unzip_data('nlp-getting-started.zip')

In [4]:
import pandas as pd

In [5]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df.head()
train_df["text"][1]


'Forest fire near La Ronge Sask. Canada'

In [6]:
train_df_shuffled = train_df.sample(frac=1,random_state=42)

In [7]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
train_df.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [9]:
len(train_df),len(test_df)

(7613, 3263)

In [10]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
    _,text,target = row
    print(f"Target:{target}","(real disaster)" if target>0 else "(not rela disaster)")
    print(f"Text :{text}")
    print('\n')
    

Target:0 (not rela disaster)
Text :@ArianaGrande @justinbieber OMGGGG IM SCREAMING


Target:0 (not rela disaster)
Text :#Kick Hendrixonfire  @'=BLACKCATS= | BIG NOOB CONQUEST | MIXED MAPS | ALL WEAPONS' for violated K/DR  Limit /Server  max 3


Target:0 (not rela disaster)
Text :@brookesddl I am traumatised the lil shit nearly hopped in the bloody shower with me


Target:1 (real disaster)
Text :Aircraft debris found on island is from MH370 Malaysia confirms http://t.co/X3RccHKagO


Target:1 (real disaster)
Text :This is one violent and belligerent storm. I'm enjoying watching it unfold




In [11]:
from sklearn.model_selection import train_test_split

train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                       train_df_shuffled["target"].to_numpy(),
                                                                       test_size = 0.1,
                                                                       random_state=42)


In [12]:
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

In [13]:
train_sentences

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       ...,
       'Near them on the sand half sunk a shattered visage lies... http://t.co/0kCCG1BT06',
       "kesabaran membuahkan hasil indah pada saat tepat! life isn't about waiting for the storm to pass it's about learning to dance in the rain.",
       "@ScottDPierce @billharris_tv @HarrisGle @Beezersun I'm forfeiting this years fantasy football pool out of fear I may win n get my ass kicked"],
      dtype=object)

In [14]:
from tensorflow.keras.layers import TextVectorization

In [15]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [16]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(
    max_tokens=max_vocab_length,
    standardize="lower_and_strip_punctuation",
    split = "whitespace",
    ngrams= None,
    output_mode = 'int',
    output_sequence_length = max_length,
    pad_to_max_tokens = True,
)


In [17]:
text_vectorizer.adapt(train_sentences)

In [18]:
sample_sentence = "hello world"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1400,  107,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]], dtype=int64)>

In [19]:
random_sentence = random.choice(train_sentences)
print(random_sentence)
text_vectorizer(random_sentence)

Drunk Meals 101: What To Cook When You're Totally Obliterated http://t.co/m19iVWrdkk


<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([1301, 2870, 2584,   55,    5, 1537,   45,  172,  802,  479,    1,
          0,    0,    0,    0], dtype=int64)>

In [20]:
words_in_vocabulary =  text_vectorizer.get_vocabulary()
top_5_words = words_in_vocabulary[:5]
bottom_5_words = words_in_vocabulary[-5:]
len(words_in_vocabulary),top_5_words,bottom_5_words

(10000,
 ['', '[UNK]', 'the', 'a', 'in'],
 ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1'])

In [21]:
# from tensorflow.keras import layers

tf.random.set_seed(42)
from tensorflow.keras import layers

embeddings = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 

embeddings

In [22]:
random_sentence = random.choice(train_sentences)
print(random_sentence)
print("\n")
print("embedded version")
embeddings(text_vectorizer([random_sentence]))

Iranian warship points weapon at U.S. helicopter official says http://t.co/SnqfHpYm3O #tcot


embedded version


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 2.09682062e-03,  2.20003016e-02,  1.07688904e-02, ...,
          2.97054648e-05, -1.31887794e-02,  3.45082991e-02],
        [-3.50671411e-02, -4.49152365e-02,  3.86092402e-02, ...,
          2.16876753e-02, -1.10107884e-02, -2.97651645e-02],
        [-3.61347198e-03,  3.27032804e-03, -8.00719112e-03, ...,
         -3.24841887e-02,  1.05396025e-02,  4.81896736e-02],
        ...,
        [ 1.01605766e-02, -3.60591300e-02,  3.03506851e-04, ...,
          1.97404139e-02,  6.34486601e-03,  3.76640595e-02],
        [ 1.01605766e-02, -3.60591300e-02,  3.03506851e-04, ...,
          1.97404139e-02,  6.34486601e-03,  3.76640595e-02],
        [ 1.01605766e-02, -3.60591300e-02,  3.03506851e-04, ...,
          1.97404139e-02,  6.34486601e-03,  3.76640595e-02]]],
      dtype=float32)>

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()),
    ("clf",MultinomialNB()),
])

model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [24]:
baseline_score=model_0.score(val_sentences,val_labels)
baseline_score

0.7926509186351706

In [25]:
baseline_preds=model_0.predict(val_sentences)

In [26]:
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1],
      dtype=int64)

In [27]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

In [28]:
def calculate_results(y_true,y_pred):
    model_accuaracy = accuracy_score(y_true,y_pred) *100
    model_precision,model_recall,model_f1,_=precision_recall_fscore_support(y_true,y_pred,average="weighted")
    model_results={
        "accuracy":model_accuaracy,
        "precision":model_precision,
        "recall":model_recall,
        "f1":model_f1
    }
    return model_results

In [29]:
baseline_results=calculate_results(val_labels,baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [30]:
from helper_functions import create_tensorboard_callback

In [31]:
SAVE_DIR='model_logs'

In [32]:
from tensorflow.keras import  layers
inputs=layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embeddings(x)
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs,name="model_1_dense")

In [33]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [34]:
model_1.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

In [35]:
model_1.history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),callbacks=[create_tensorboard_callback(SAVE_DIR,'model_1_dense')])

Saving TensorBoard log files to: model_logs/model_1_dense/20240911-235149
Epoch 1/5
215/215 [==============================] - 3s 7ms/step - loss: 0.6091 - accuracy: 0.6920 - val_loss: 0.5356 - val_accuracy: 0.7546
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.4407 - accuracy: 0.8180 - val_loss: 0.4693 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.3461 - accuracy: 0.8608 - val_loss: 0.4593 - val_accuracy: 0.7927
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2846 - accuracy: 0.8927 - val_loss: 0.4646 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.2378 - accuracy: 0.9123 - val_loss: 0.4770 - val_accuracy: 0.7861


In [36]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [37]:
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4770 - accuracy: 0.7861


[0.4770491421222687, 0.7860892415046692]

In [38]:
model_1_pred_probs=model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 3ms/step


(762, 1)

In [39]:
model_1_pred_probs[0]

array([0.40672264], dtype=float32)

In [40]:
model_1_pred_probs

array([[4.06722635e-01],
       [7.45447576e-01],
       [9.98010457e-01],
       [1.08101368e-01],
       [1.13053359e-01],
       [9.34729457e-01],
       [9.16833639e-01],
       [9.93083596e-01],
       [9.70149040e-01],
       [2.65630931e-01],
       [1.20250702e-01],
       [7.04488277e-01],
       [5.23457527e-02],
       [2.19779238e-01],
       [5.21695195e-03],
       [1.58112988e-01],
       [2.73795165e-02],
       [1.03364982e-01],
       [2.41125345e-01],
       [5.54777324e-01],
       [9.10140157e-01],
       [4.29593064e-02],
       [4.42966551e-01],
       [1.02871120e-01],
       [9.57128942e-01],
       [9.98998225e-01],
       [3.32367122e-02],
       [7.00706020e-02],
       [2.62654107e-02],
       [2.11718783e-01],
       [5.83187044e-01],
       [2.35835567e-01],
       [5.09473264e-01],
       [2.06295684e-01],
       [5.28980076e-01],
       [5.51430732e-02],
       [9.94448185e-01],
       [1.41781196e-01],
       [3.55752483e-02],
       [9.98508394e-01],


In [41]:
model_1_pred_probs[:10] 

array([[0.40672264],
       [0.7454476 ],
       [0.99801046],
       [0.10810137],
       [0.11305336],
       [0.93472946],
       [0.91683364],
       [0.9930836 ],
       [0.97014904],
       [0.26563093]], dtype=float32)

In [42]:
model_1_preds=tf.squeeze(tf.round(model_1_pred_probs))

In [43]:
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [44]:
model_1_results = calculate_results(val_labels,model_1_preds)

In [45]:
model_1_results

{'accuracy': 78.60892388451444,
 'precision': 0.7903277546022673,
 'recall': 0.7860892388451444,
 'f1': 0.7832971347503846}

In [46]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [47]:
import numpy as np 
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

In [48]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab),words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [49]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [50]:
embed_weights = model_1.get_layer("embedding_1").get_weights()[0]

In [51]:
embed_weights

array([[ 0.02537496, -0.02235664,  0.01426724, ...,  0.03417593,
         0.02083804,  0.05248006],
       [ 0.04635455,  0.03081604, -0.00414347, ..., -0.04626253,
         0.03793519, -0.03401703],
       [-0.00978814,  0.05545923,  0.05595186, ..., -0.00938332,
         0.0477787 ,  0.02449379],
       ...,
       [-0.00880691,  0.00455992,  0.0274366 , ..., -0.04172378,
        -0.0253644 , -0.03372788],
       [ 0.00634911,  0.01384877,  0.09206314, ...,  0.02170778,
         0.05075679,  0.07815135],
       [ 0.05306496,  0.06554126,  0.08889391, ...,  0.11545217,
         0.09476621,  0.02966497]], dtype=float32)

In [52]:
embed_weights.shape

(10000, 128)

In [53]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [54]:
from tensorflow.keras import  layers

In [55]:
inputs = layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embeddings(x)
# x=layers.LSTM(64,return_sequences=True)(x)
x=layers.LSTM(64)(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model_2=tf.keras.Model(inputs,outputs,name="model_2_LTSM")

 

In [56]:
model_2.summary()

Model: "model_2_LTSM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [57]:
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [58]:
model_2.fit(train_sentences,train_labels,epochs=5,validation_data=(val_sentences,val_labels),callbacks=[create_tensorboard_callback(SAVE_DIR,"model_2_LTSM")])

Saving TensorBoard log files to: model_logs/model_2_LTSM/20240911-235159
Epoch 1/5
215/215 [==============================] - 8s 14ms/step - loss: 0.2190 - accuracy: 0.9242 - val_loss: 0.5372 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1608 - accuracy: 0.9418 - val_loss: 0.6074 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1265 - accuracy: 0.9518 - val_loss: 0.7453 - val_accuracy: 0.7822
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.1049 - accuracy: 0.9604 - val_loss: 0.8508 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0781 - accuracy: 0.9672 - val_loss: 1.0004 - val_accuracy: 0.7822


In [59]:
model_2_pred_probs = model_2.predict(val_sentences)

24/24 [==============================] - 1s 4ms/step


In [60]:
model_1_pred_probs[:10]

array([[0.40672264],
       [0.7454476 ],
       [0.99801046],
       [0.10810137],
       [0.11305336],
       [0.93472946],
       [0.91683364],
       [0.9930836 ],
       [0.97014904],
       [0.26563093]], dtype=float32)

In [61]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [62]:
model_2_results = calculate_results(val_labels,model_2_preds)

In [63]:
model_2_results

{'accuracy': 78.21522309711287,
 'precision': 0.7849731976577976,
 'recall': 0.7821522309711286,
 'f1': 0.7797935460636591}

In [64]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [68]:
inputs = layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embeddings(x)
# x=layers.GRU(64,return_sequences=True)(x)
# x=layers.LSTM(64,return_sequences=True)(x)
# x=layers.GRU(64)(x)
# x=layers.GlobalAveragePooling1D()(x)
x=layers.GRU(64)(x)
x=layers.Dense(64,activation="relu")(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_3=tf.keras.Model(inputs,outputs,name="model_3_GRU")




In [69]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                       

In [70]:
model_3.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [71]:
model_3_history = model_3.fit(train_sentences,train_labels,epochs=5,validation_data=(val_sentences,val_labels),callbacks=[create_tensorboard_callback(SAVE_DIR,"model_3_gru")])

Saving TensorBoard log files to: model_logs/model_3_gru/20240911-235333
Epoch 1/5
215/215 [==============================] - 5s 12ms/step - loss: 0.1589 - accuracy: 0.9453 - val_loss: 0.6743 - val_accuracy: 0.7822
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0847 - accuracy: 0.9695 - val_loss: 0.8445 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0667 - accuracy: 0.9742 - val_loss: 1.1905 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0566 - accuracy: 0.9759 - val_loss: 1.1522 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0503 - accuracy: 0.9772 - val_loss: 1.3907 - val_accuracy: 0.7730


In [72]:
model_3_pred_probs=model_3.predict(val_sentences)
model_1_pred_probs

24/24 [==============================] - 1s 5ms/step


array([[4.06722635e-01],
       [7.45447576e-01],
       [9.98010457e-01],
       [1.08101368e-01],
       [1.13053359e-01],
       [9.34729457e-01],
       [9.16833639e-01],
       [9.93083596e-01],
       [9.70149040e-01],
       [2.65630931e-01],
       [1.20250702e-01],
       [7.04488277e-01],
       [5.23457527e-02],
       [2.19779238e-01],
       [5.21695195e-03],
       [1.58112988e-01],
       [2.73795165e-02],
       [1.03364982e-01],
       [2.41125345e-01],
       [5.54777324e-01],
       [9.10140157e-01],
       [4.29593064e-02],
       [4.42966551e-01],
       [1.02871120e-01],
       [9.57128942e-01],
       [9.98998225e-01],
       [3.32367122e-02],
       [7.00706020e-02],
       [2.62654107e-02],
       [2.11718783e-01],
       [5.83187044e-01],
       [2.35835567e-01],
       [5.09473264e-01],
       [2.06295684e-01],
       [5.28980076e-01],
       [5.51430732e-02],
       [9.94448185e-01],
       [1.41781196e-01],
       [3.55752483e-02],
       [9.98508394e-01],


In [89]:
model_3_preds = tf.squeeze(tf.round(model_1_pred_probs))

In [90]:
model_3_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0

In [91]:
model_3_results=calculate_results(val_labels,model_3_preds)

In [92]:
model_3_results

{'accuracy': 78.60892388451444,
 'precision': 0.7903277546022673,
 'recall': 0.7860892388451444,
 'f1': 0.7832971347503846}

In [81]:
inputs = layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embeddings(x)
# x=layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x)
x=layers.Bidirectional(layers.LSTM(64))(x)
# x=layers.Dense(64,activation="relu")(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_4=tf.keras.Model(inputs,outputs,name="model_4_bidirectional")

In [82]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense_5 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,378,945
Trainable params: 1,3

In [83]:
model_4.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [84]:
model_4_history = model_4.fit(train_sentences,train_labels,epochs=5,validation_data=(val_sentences,val_labels),callbacks=[create_tensorboard_callback(SAVE_DIR,"model_3_gru")])

Saving TensorBoard log files to: model_logs/model_3_gru/20240912-000912
Epoch 1/5
215/215 [==============================] - 9s 19ms/step - loss: 0.1146 - accuracy: 0.9685 - val_loss: 0.9431 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0543 - accuracy: 0.9758 - val_loss: 1.1775 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0467 - accuracy: 0.9784 - val_loss: 1.3116 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0421 - accuracy: 0.9788 - val_loss: 1.3369 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0418 - accuracy: 0.9801 - val_loss: 1.3705 - val_accuracy: 0.7677


In [85]:
model_4_pred_probs=model_4.predict(val_sentences)

24/24 [==============================] - 1s 5ms/step


In [86]:
model_4_preds=tf.squeeze(tf.round(model_4_pred_probs))

In [87]:
model_4_preds

<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 0.,
       1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0

In [93]:
model_4_results=calculate_results(val_labels,model_4_preds)

In [94]:
model_4_results

{'accuracy': 76.77165354330708,
 'precision': 0.7675450859410361,
 'recall': 0.7677165354330708,
 'f1': 0.7667932666650168}

In [97]:
embedding_test=embeddings(text_vectorizer(["This is a demo text "]))
conv_1d=layers.Conv1D(
    filters=32,
    kernel_size=4,
    activation="relu",
    padding="valid"
)
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPooling1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape , conv_1d_output.shape,max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 12, 32]), TensorShape([1, 32]))

In [98]:
embedding_test

<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00974727,  0.0704824 ,  0.04858844, ..., -0.07390584,
          0.00191417,  0.01954197],
        [ 0.02099935,  0.01786441,  0.00832643, ..., -0.00743995,
          0.04569955,  0.01522546],
        [-0.02758015,  0.07832696, -0.00952926, ..., -0.01533029,
          0.05373053, -0.02491544],
        ...,
        [ 0.01894247,  0.01172048,  0.02119346, ..., -0.00971636,
          0.0254826 ,  0.03159623],
        [ 0.01894247,  0.01172048,  0.02119346, ..., -0.00971636,
          0.0254826 ,  0.03159623],
        [ 0.01894247,  0.01172048,  0.02119346, ..., -0.00971636,
          0.0254826 ,  0.03159623]]], dtype=float32)>

In [99]:
conv_1d_output

<tf.Tensor: shape=(1, 12, 32), dtype=float32, numpy=
array([[[0.        , 0.        , 0.02724525, 0.0423285 , 0.05742417,
         0.0174136 , 0.01026981, 0.00342176, 0.        , 0.00638254,
         0.        , 0.06769368, 0.08513728, 0.        , 0.        ,
         0.04336627, 0.        , 0.0519232 , 0.        , 0.0313793 ,
         0.00116136, 0.01897326, 0.03719107, 0.        , 0.0018488 ,
         0.        , 0.01484968, 0.        , 0.        , 0.01007323,
         0.02106364, 0.        ],
        [0.27845946, 0.18563843, 0.33847827, 0.30999577, 0.        ,
         0.06571174, 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.13089968, 0.1822504 , 0.16441081,
         0.04530699, 0.        , 0.        , 0.        , 0.11117437,
         0.        , 0.        , 0.        , 0.40344852, 0.3018574 ,
         0.01187279, 0.35401466],
        [0.        , 0.15626651, 0.16895331, 0.    

In [103]:
inputs = layers.Input(shape=(1,),dtype=tf.string)
x=text_vectorizer(inputs)
x=embeddings(x)
x=layers.Conv1D(filters=32,
                kernel_size=5,
                strides=1,
                activation="relu",
                padding="valid")(x)
x=layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_5=tf.keras.Model(inputs,outputs,name="model_5_Conv1D")

In [104]:
model_5.compile(loss='binary_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

In [105]:
model_5_history = model_4.fit(train_sentences,train_labels,epochs=5,validation_data=(val_sentences,val_labels),callbacks=[create_tensorboard_callback(SAVE_DIR,"model_3_gru")])


Saving TensorBoard log files to: model_logs/model_3_gru/20240912-003256
Epoch 1/5
215/215 [==============================] - 4s 12ms/step - loss: 0.0407 - accuracy: 0.9809 - val_loss: 1.4318 - val_accuracy: 0.7677
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0340 - accuracy: 0.9812 - val_loss: 1.5700 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0336 - accuracy: 0.9823 - val_loss: 1.7908 - val_accuracy: 0.7625
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0379 - accuracy: 0.9806 - val_loss: 1.3345 - val_accuracy: 0.7703
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0326 - accuracy: 0.9831 - val_loss: 1.7164 - val_accuracy: 0.7717


In [107]:
model_5_pred_probs = model_5.predict(val_sentences)

24/24 [==============================] - 0s 3ms/step


In [108]:
model_5_preds=tf.squeeze(tf.round(model_5_pred_probs))

In [109]:
model_5_results = calculate_results(val_labels,model_5_preds)

In [112]:
model_5_results

{'accuracy': 55.90551181102362,
 'precision': 0.5547271951753044,
 'recall': 0.5590551181102362,
 'f1': 0.5542809125143942}